# Mission to Mars

### NASA Mars News

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/)

In [1]:
#import dependencies
from bs4 import BeautifulSoup as BS
from splinter import Browser
import os
import pandas as pd

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url_news = "https://mars.nasa.gov/news/"
browser.visit(url_news)

# Parse HTML with Beautiful Soup
html_news = browser.html
news_soup = BS(html_news, "html.parser")
# print(soup.prettify())

In [5]:
# Collect the latest News Title and Paragraph Text
news_title = news_soup.find("div",class_="content_title").text
news_p = news_soup.find("div", class_="article_teaser_body").text
print(f"Latest Title: {news_title}")
print(f"Paragraph: {news_p}")

Latest Title: Robotic Toolkit Added to NASA's Mars 2020 Rover
Paragraph: The bit carousel, which lies at the heart of the rover's Sample Caching System, is now aboard NASA's newest rover. 


### JPL Mars Space Images - Featured Image

In [6]:
url_img = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_img)

In [7]:
##################
# Option1: Directly scraping with Chrome
html_img = browser.html
image_soup = BS(html_img, "html.parser")
img_url_opt1 = image_soup.find_all('a', class_ = 'fancybox')[1]["data-fancybox-href"]

In [8]:
##################
html_img = browser.html
# Option2 (Preferred): Design an xpath selector to grab the image
xpath = "//*[@id=\"page\"]/section[3]/div/ul/li[1]/a/div/div[2]/img"

#Use splinter to click to bring the full resolution image
img = browser.find_by_xpath(xpath)[0]
img.click()

In [9]:
html_img = browser.html
image_soup = BS(html_img, "html.parser")
img_url_opt2 = image_soup.find("img", class_="fancybox-image")["src"]

In [11]:
# Get base url
from urllib.parse import urlsplit
base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url_img))

featured_img_url = base_url + img_url_opt2
print (featured_img_url)

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA23384_hires.jpg


### Mars Weather

In [12]:
# Visit the Mars Weather twitter account and scrape the latest Mars weather tweet from the page
url_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_weather)

html_weather = browser.html
weather_soup = BS(html_weather, "html.parser")

# Get latest Mars weather tweet
tweets = weather_soup.find_all("div",class_="js-tweet-text-container")[0].text
mars_weather = tweets[9:-27].replace('\n','')

print (mars_weather)

sol 258 (2019-08-18) low -100.0ºC (-148.1ºF) high -26.2ºC (-15.2ºF)winds from the SSE at 5.3 m/s (11.9 mph) gusting to 16.8 m/s (37.6 mph)pressure at 7.60 hPa


### Mars Facts

In [13]:
# Scrape the table containing Mars Planet Profile
url_mars_facts = "https://space-facts.com/mars/"
table_df = pd.read_html(url_mars_facts)[1]
table_df.columns = ["Description", "Values"]
table_df_new = table_df.set_index(["Description"])
table_df_new

,Values
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [14]:
# Use Pandas to convert the data to a HTML table string.
html_mars_facts = table_df_new.to_html().replace("\n","")
html_mars_facts

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Values</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

### Mars Hemispheres

In [15]:
url_hemispheres = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

# Find base url 
url_hemispheres_base  = "{0.scheme}://{0.netloc}/".format(urlsplit(url_hemispheres))

browser.visit(url_hemispheres)

In [16]:
hemisphere_image_urls = []

for x in range(1,5):  # Four results in total
    # click each of the links
    products = browser.find_by_xpath( "//*[@id='product-section']/div[2]/div["+str(x)+"]/a/img").click()
    full_img = browser.find_by_xpath( "//*[@id='wide-image-toggle']").click()
    # Parse HTML with Beautiful Soup
    image = browser.html
    soup = BS(image, "html.parser")
    url = soup.find("img", class_="wide-image")["src"]
    full_url = url_hemispheres_base + url
    title = soup.find("h2",class_="title").text
    # Append the dictionary with the image url string and the hemisphere title to a list
    hemisphere_image_urls.append({"title": title, "img_url": full_url})
    # Going back
    browser.visit(url_hemispheres)

In [17]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]